# Game of War (cards) Simulation #
Simulate the game of War (card game).

Ths notebook will simulate playing the card came War.  It will play it _n_ number of times and report basic statistics at the end:  number of games played, number of hands played, number of wins by each player (would expect this to be roughly 50-50), and the number of wars waged.


Number of players:  2
Objective: Collect all of the cards in a standard card deck (excluding Jokers).
Rules:
  1.  The entire deck of cards is dealt in an alternating fashion to each player (each player recieves 26 cards).
  2.  Player 1 lays a card from the top of thier deck face up on the table.  Player 2 does the same.
  3.  The card with the highest value wins the "battle". The winning player collects all of the cards and sets them aside, face down.  (Cards are valued 2 - A with 2 the lowest card and ace the highest.)
  4.  This process continues until each player lays down a card of equal value.  This is a war!
  5.  Procedures for a war:
      a.  Each player lays down three cards face down.
      b.  The fourth card is dealt face up.  The card with the highest value (same as rule #3) wins the war and collects all of the cards.
      c.  It is possible that the fourth cards dealt could be of the same value, which incites an additional war.  In this case the process repeats (steps a and b).  The winner takes all of the cards.
      d.  If a player does not have enough cards to complete the war (4 cards), they lose and the other player wins by default.
  6.  When a player exhausts thier deck of cards, thier discard deck is played (without shuffling).


In [ ]:
import random

games_to_play = 100  # change this value to play more or fewer games

suits = ['H', 'D', 'S', 'C']
cards = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
deck = []
player1 = []
player2 = []

In [ ]:
def create_deck():
    """
    Create a list representing a deck of shuffled cards.  Individual cards are represented as suit followed by value:  
      H2 = two of hearts,
      S10 = ten of spades,
      DQ = queen of diamonds,
      CA = ace of clubs, etc.
      
      Returns
      -------
      card_deck : list - representing a shuffled deck of cards
    """
    card_deck = []
    for suit in suits:
        for card in cards:
            card_deck.append(suit + card)
    random.seed()        
    random.shuffle(card_deck)
    return card_deck
  

def deal_cards(deck):
    """
    Deal cards to each player.
    
    Parameters
    ----------
    deck : list - representing a shuffled deck of cards
    
    Returns
    -------
    p1, p2 : list - representing hands dealt to player 1 and player 2
    """
    p1 = []
    p2 = []
    
    for i in range(len(deck)):
        if i % 2 == 0:
            p1.append(deck[i])
        else:
            p2.append(deck[i])

    return p1, p2
    

def get_card_value(card):
    """
    Determine the value of a card for comparison purposes.  The value is determined by finding the index of the card's value
    in the cards[] list.  The suit is meaningless in War, so only the value portion of the card is used.
    
    Parameters
    ----------
    card : str - represents a card.  For example: 'SK' (king of spades), 'D3' (three of diamonds)
    
    Returns
    -------
    int - index of card value in cards[] list
    """
    return cards.index(str(card)[1:])


def play_a_game(player1, player2):
    """
    Play a single game of War.  In a loop, each player plays thier [0] card and compares their values to determine the
    "battle" winner.  The loop continues while both players have cards (size of thier deck is greater than 0).  As soon as
    one player losses all thier cards, the game ends.
    
    If the cards played are equal, the goto_war() function is called.
    
    If player 1 wins, he collects all the played cards.  If player 2 wins, she collects all the played cards.  This
    is achieved by appending the loser's card to the end of the winner's deck.  Then, appending the winner's card the end
    of his/her own deck.  These two steps effectively put the cards in the discard pile.  Finally, the [0] card is removed
    from each player's hand.
    
    Parameters
    ----------
    player1, player2 : list - representing cards to play
    
    Returns
    -------
    hands : int - the number of "battles"
    wars : int - the number of wars waged
    player1, player2 : list - players hands at the end of the game.  One of these should have length 0 and the other 52
    """
    
    hands = 0
    wars = 0
    
    # play until one player does not have any cards left
    while len(player1) > 0 and len(player2) > 0:
        hands += 1
        # print('----- hand', hands, '-----')
        # print('player1=', player1[0], ': player2=', player2[0])
        
        # WAR!
        if get_card_value(player1[0]) == get_card_value(player2[0]):
            # print('** WAR! **')
            winner, player1, player2, wars = goto_war(player1, player2, wars)
            
        # player1 wins
        elif get_card_value(player1[0]) > get_card_value(player2[0]):
            # put player2 card at end of player1 deck
            player1.append(player2[0])
            # remove player2 card
            player2 = player2[1:]
            # move player1 card to end of deck
            player1.append(player1[0])
            # remove played card from top of deck
            player1 = player1[1:]
            # print('player1 wins hand')
        
        # player2 wins
        else:
            # put player1 card at end of player2 deck
            player2.append(player1[0])
            # remove player1 card
            player1 = player1[1:]
            # move player2 card to end of deck
            player2.append(player2[0])
            # remove played card from top of deck
            player2 = player2[1:]            
            # print('player2 wins hand')
            
        # print('Player1 deck =', player1)
        # print('player2 deck =', player2)
        
    return hands, wars, player1, player2


def goto_war(player1, player2, wars):
    """
    Implement the war.  If any player posesses fewer than four cards they default and the other player wins.  If both
    players are able to "fight", then each player removes four cards from thier deck.  The value of the fourth card
    determines the winner of the battle.  Note this function is recursive!  It is possible to have multi-layered wars.
    
    Parameters
    ----------
    player1, player2 : list - current player decks of cards
    wars : int - number of wars fought
    
    Returns
    -------
    winner : str - the winner of the war
    player1, player2 : list - player decks after the war
    wars : int - updated number of wars fought
    
    """
    winner = ''
    wars += 1
    
    # goto war
    if len(player1) > 4 and len(player2) > 4:
        
        # deal off three cards and remove from decks
        player1_war_cards = player1[0:5]
        player1 = player1[5:]
        # print('player1 war cards', player1_war_cards)
        # print('player1=', player1)
        player2_war_cards = player2[0:5]
        player2 = player2[5:]
        # print('player2 war cards', player2_war_cards)
        # print('player2=', player2)
        
        player1_war_card = player1_war_cards[4]
        player2_war_card = player2_war_cards[4]
        # print('player1 war card=', player1_war_card)
        # print('player2 war card=', player2_war_card)
        
        # WAR again!
        if get_card_value(player1_war_card) == get_card_value(player2_war_card):
            # print('** WAR Again! **')
            winner, player1, player2, wars = goto_war(player1, player2, wars)
            
            # account for cards sacraficed in the parent war
            if winner == 'player1':
                # print('player1 wins multi-layer war')
                player1.extend(player1_war_cards)
                player1.extend(player2_war_cards)
            else:
                # print('player2 wins multi-layer war')
                player2.extend(player2_war_cards)
                player2.extend(player1_war_cards)
            
        # player1 wins war
        elif get_card_value(player1_war_card) > get_card_value(player2_war_card):
            # print('player1 wins war')
            player1.extend(player1_war_cards)
            player1.extend(player2_war_cards)
            winner = 'player1'
        
        # player2 wins war
        else:
            # print('player2 wins war')
            player2.extend(player2_war_cards)
            player2.extend(player1_war_cards)
            winner = 'player2'  
    
    # player1 wins by default
    elif len(player2) < 4:
        # print('player1 wins by default')
        player1.extend(player2)
        player2 = []
        winner = 'player1'
        
    # player2 wins by default
    else:
        # print('player2 wins by default')
        player2.extend(player1)    
        player1 = []
        winner = 'player2'
    
    # print('player1=', player1)
    # print('player2=', player2)
    # print('winner=', winner)
    
    return winner, player1, player2, wars


In [ ]:
"""
Main processing loop
"""

# init aggregate varibles
i = 0
p1_tot = 0
p2_tot = 0
h = 0
w = 0

while i in range(0, games_to_play):
    i += 1

    deck = create_deck()
    player1, player2 = deal_cards(deck)
    hands, wars, player1, player2 = play_a_game(player1, player2)
    
    print('===== GAME', i, '=====')

    if len(player1) > len(player2):
        print('Player 1 wins game')
        p1_tot += 1
    else:
        print('Player 2 wins game')
        p2_tot += 1
        
    print(hands, 'Hands played')
    h = h + hands
    
    print(wars, 'Wars Waged')
    w = w + wars
    
print('=============================')
print('Games played: ', games_to_play)
print('Player 1 wins: ', p1_tot)
print('Player 2 wins: ', p2_tot)
print('Total hands played: ', h)
print('Total wars waged: ', w)
print('=============================')

# <SDG><
